# Foundations of Data Engineering with SQL


Welcome! This hands-on course spans **5 sessions** and a **capstone project**.  
You'll use **SQLite** (built into Python) for portable, zero-setup SQL practice.  
All content here is original and provided for educational use.

> If you're viewing this in Google Colab, you can run each cell with Shift+Enter.  
> If you're local, use `jupyter notebook` or VS Code to open and run cells.

## Environment Setup

In [ ]:
# No external installs needed. We'll use Python's built-in sqlite3.
import sqlite3, pandas as pd, numpy as np
from pathlib import Path

DB_PATH = Path('course.db')  # SQLite database file in the current working directory
if DB_PATH.exists():
    DB_PATH.unlink()  # start fresh on re-run for deterministic exercises

conn = sqlite3.connect(DB_PATH)
conn.execute("PRAGMA foreign_keys = ON;")
print("SQLite database created at", DB_PATH.resolve())

### Helper: run_sql

In [ ]:
def run_sql(q, params=None, limit=None):
    """Convenience: run a SQL string and display as a DataFrame."""
    params = params or {}
    df = pd.read_sql_query(q, conn, params=params)
    if limit is not None:
        display(df.head(limit))
    else:
        display(df)
    return df

## Session 1 — Data Engineering Overview + First SQL

**Objectives**
- Understand the data engineering lifecycle (ingest → clean → transform → store → serve).
- See where SQL is used in wrangling and transformation.
- Run first queries against small, in-notebook tables.

**Concepts**
- Tables, rows, columns
- SELECT, ORDER BY

In [ ]:
# Create two tiny starter tables
conn.executescript('''
CREATE TABLE customers (
    customer_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    city TEXT,
    country TEXT,
    address TEXT
);
INSERT INTO customers (name, city, country, address) VALUES
  ('Anders', 'Berlin', 'Germany', 'Alexanderplatz 1'),
  ('Bella', 'Bern', 'Switzerland', 'Bahnhofstrasse 5'),
  ('Chen', 'Boston', 'USA', '1 Main St'),
  ('Dario', 'Dallas', 'USA', '2 Elm St'),
  ('Elena', 'Berlin', 'Germany', NULL);

CREATE TABLE products (
    product_id INTEGER PRIMARY KEY,
    product TEXT NOT NULL,
    price REAL NOT NULL CHECK(price >= 0)
);
INSERT INTO products (product, price) VALUES
  ('Keyboard', 39.99), ('Mouse', 24.50), ('Monitor', 199.00), ('USB-C Cable', 8.99);
''')
conn.commit()
print("Seed tables created.")

In [ ]:
# First queries
run_sql("SELECT * FROM customers ORDER BY city, name;")
run_sql("SELECT name, price FROM products ORDER BY price DESC;")

**Try it:** Change the `ORDER BY` column, or select a subset of columns.

## Session 2 — Modeling & Data Types

**Objectives**
- Learn keys and relationships.
- Create an OLTP-style mini schema (customers, orders, order_items, shippers).

**Concepts**
- PRIMARY KEY, FOREIGN KEY
- Data types & constraints

In [ ]:
conn.executescript('''
CREATE TABLE shippers (
    shipper_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL
);

CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY,
    customer_id INTEGER NOT NULL,
    order_date TEXT NOT NULL,
    shipper_id INTEGER,
    FOREIGN KEY(customer_id) REFERENCES customers(customer_id),
    FOREIGN KEY(shipper_id) REFERENCES shippers(shipper_id)
);

CREATE TABLE order_items (
    order_item_id INTEGER PRIMARY KEY,
    order_id INTEGER NOT NULL,
    product_id INTEGER NOT NULL,
    quantity INTEGER NOT NULL CHECK (quantity > 0),
    FOREIGN KEY(order_id) REFERENCES orders(order_id),
    FOREIGN KEY(product_id) REFERENCES products(product_id)
);

INSERT INTO shippers (name) VALUES ('Postal Express'), ('GoFast'), ('Eagle Air');
INSERT INTO orders (customer_id, order_date, shipper_id) VALUES
  (1, '2024-10-01', 1), (1, '2024-10-05', 2), (3, '2024-11-01', 2), (4, '2024-11-03', 3);
INSERT INTO order_items (order_id, product_id, quantity) VALUES
  (1, 1, 2), (1, 4, 3),
  (2, 3, 1),
  (3, 2, 1), (3, 4, 2),
  (4, 3, 2);
''')
conn.commit()
print("Transactional schema created and populated.")

In [ ]:
# Inspect schema via PRAGMA and sample joins
run_sql("PRAGMA table_info('orders');")
run_sql('''SELECT o.order_id, c.name AS customer, o.order_date, s.name AS shipper
          FROM orders o
          LEFT JOIN customers c ON o.customer_id=c.customer_id
          LEFT JOIN shippers s ON o.shipper_id=s.shipper_id
          ORDER BY o.order_date;''')

## Session 3 — Profiling & Cleaning Data

**Objectives**
- Profile distributions and detect common quality issues.
- Practice `COUNT`, `MIN`, `MAX`, `AVG`, NULL checks, LIKE patterns.

**Real scenario**
- Identify missing addresses, cities with low coverage, and price outliers.

In [ ]:
# Profiling examples
run_sql("SELECT COUNT(*) AS n_customers FROM customers;")
run_sql("SELECT MIN(price) AS min_price, MAX(price) AS max_price, AVG(price) AS avg_price FROM products;")
run_sql("SELECT * FROM customers WHERE address IS NULL;")
run_sql("SELECT * FROM customers WHERE city LIKE 'B%';")

In [ ]:
# Simple outlier detection using z-score on product price (for teaching purposes)
dfp = run_sql("SELECT product_id, product, price FROM products;")
mu, sigma = dfp["price"].mean(), dfp["price"].std(ddof=0)
dfp["z"] = (dfp["price"] - mu) / (sigma if sigma else 1.0)
display(dfp)
print("Prices with |z| >= 2 considered potential outliers in this simple demo:")
display(dfp.loc[dfp["z"].abs() >= 2])

**Exercise:** Write a query to list cities and how many customers live in each; order by count desc.

## Session 4 — Transformation: Joins, Grouping, Pivoting

**Objectives**
- Combine tables with joins.
- Summarize with `GROUP BY` and filter aggregates with `HAVING`.
- Demonstrate a pivot (emulated in SQLite using conditional aggregation).

In [ ]:
# Join + aggregation: revenue per customer
run_sql('''
SELECT c.name AS customer,
       SUM(oi.quantity * p.price) AS revenue
FROM orders o
JOIN order_items oi ON o.order_id = oi.order_id
JOIN products p ON oi.product_id = p.product_id
JOIN customers c ON o.customer_id = c.customer_id
GROUP BY c.name
ORDER BY revenue DESC;
''')

In [ ]:
# Pivot style using conditional aggregation: product count by shipper
run_sql('''
SELECT s.name AS shipper,
       SUM(CASE WHEN p.product='Keyboard' THEN oi.quantity ELSE 0 END) AS keyboard_qty,
       SUM(CASE WHEN p.product='Mouse' THEN oi.quantity ELSE 0 END) AS mouse_qty,
       SUM(CASE WHEN p.product='Monitor' THEN oi.quantity ELSE 0 END) AS monitor_qty,
       SUM(CASE WHEN p.product='USB-C Cable' THEN oi.quantity ELSE 0 END) AS cable_qty
FROM orders o
JOIN shippers s ON o.shipper_id = s.shipper_id
JOIN order_items oi ON o.order_id = oi.order_id
JOIN products p ON oi.product_id = p.product_id
GROUP BY s.name
ORDER BY s.name;
''')

**Exercise:** Create a monthly sales summary (YYYY-MM) with total revenue and order count.

## Session 5 — DDL, DML, and Transactions

**Objectives**
- Create/alter tables (DDL), insert/update/delete data (DML).
- Use transactions (`BEGIN`, `COMMIT`, `ROLLBACK`) safely.
- Practice subqueries.

**Note:** SQLite auto-commits by default; we use explicit transactions for teaching.

In [ ]:
# DDL: add a column
conn.execute("ALTER TABLE customers ADD COLUMN loyalty_tier TEXT DEFAULT 'Standard';")
conn.commit()
run_sql("SELECT * FROM customers;")

In [ ]:
# Transaction demo
print("Before:")
run_sql("SELECT order_id, SUM(quantity) AS total_qty FROM order_items GROUP BY order_id ORDER BY order_id;")
try:
    conn.execute("BEGIN;")
    conn.execute("UPDATE order_items SET quantity = quantity + 1 WHERE order_id IN (1,2);")
    print("During (uncommitted):")
    run_sql("SELECT order_id, SUM(quantity) AS total_qty FROM order_items GROUP BY order_id ORDER BY order_id;")
    # intentionally roll back for demo
    conn.execute("ROLLBACK;")
finally:
    pass

print("After rollback:")
run_sql("SELECT order_id, SUM(quantity) AS total_qty FROM order_items GROUP BY order_id ORDER BY order_id;")

In [ ]:
# Subquery example: orders with item count above average
run_sql('''
SELECT o.order_id, COUNT(*) AS n_items
FROM order_items oi
JOIN orders o ON oi.order_id = o.order_id
GROUP BY o.order_id
HAVING n_items > (SELECT AVG(cnt) FROM (
    SELECT COUNT(*) AS cnt FROM order_items GROUP BY order_id
) t);
''')

## Capstone — Mini Data Warehouse for Real Estate Analytics

**Goal:** Build an end-to-end SQL pipeline to analyze home sales.

**Steps**
1. Design a schema: `locations(city, state)`, `agents`, `homes`, `sales`.
2. Create and populate tables with synthetic data.
3. Profile data for gaps and outliers.
4. Clean / standardize (e.g., unify state names).
5. Build analytics queries (avg/median price per city, top agents, YOY trends).
6. (Optional) Add a pivot-style summary for city × year.

**Deliverables (export as .sql or .ipynb section)**
- DDL + inserts
- Profiling/cleaning SQL
- Final analytics SQL with short narrative

In [ ]:
# Starter capstone schema with synthetic data
conn.executescript('''
CREATE TABLE locations (
    location_id INTEGER PRIMARY KEY,
    city TEXT NOT NULL,
    state TEXT NOT NULL
);

CREATE TABLE agents (
    agent_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL
);

CREATE TABLE homes (
    home_id INTEGER PRIMARY KEY,
    location_id INTEGER NOT NULL,
    bedrooms INTEGER NOT NULL,
    bathrooms REAL NOT NULL,
    sqft INTEGER NOT NULL,
    home_type TEXT NOT NULL,
    year_built INTEGER,
    FOREIGN KEY(location_id) REFERENCES locations(location_id)
);

CREATE TABLE sales (
    sale_id INTEGER PRIMARY KEY,
    home_id INTEGER NOT NULL,
    agent_id INTEGER NOT NULL,
    sale_date TEXT NOT NULL,
    sale_price REAL NOT NULL CHECK (sale_price >= 0),
    FOREIGN KEY(home_id) REFERENCES homes(home_id),
    FOREIGN KEY(agent_id) REFERENCES agents(agent_id)
);

INSERT INTO locations (city, state) VALUES
  ('Austin', 'TX'), ('Austin', 'Texas'), ('Boston', 'MA'), ('Dallas', 'TX'), ('Denver', 'CO');

INSERT INTO agents (name) VALUES ('Alex Kim'), ('Priya Singh'), ('Carlos M.');

INSERT INTO homes (location_id, bedrooms, bathrooms, sqft, home_type, year_built) VALUES
  (1, 3, 2.0, 1600, 'SingleFamily', 2002),
  (2, 4, 2.5, 2200, 'SingleFamily', 2010),
  (3, 2, 1.0, 900,  'Condo', 1995),
  (4, 3, 2.0, 1400, 'Townhome', 2008),
  (5, 5, 3.0, 2800, 'SingleFamily', 2018);

INSERT INTO sales (home_id, agent_id, sale_date, sale_price) VALUES
  (1, 1, '2024-05-11', 420000),
  (2, 2, '2024-06-20', 630000),
  (3, 2, '2024-03-03', 390000),
  (4, 3, '2024-09-01', 415000),
  (5, 1, '2024-02-14', 780000);
''')
conn.commit()
print("Capstone base schema seeded.")

In [ ]:
# Capstone tasks: profiling, cleaning, analytics
print("Profiling — inconsistent state names:")
run_sql("SELECT state, COUNT(*) AS n FROM locations GROUP BY state ORDER BY n DESC;")

print("Cleaning — unify 'Texas' -> 'TX':")
conn.execute("UPDATE locations SET state='TX' WHERE state LIKE 'Texas';")
conn.commit()
run_sql("SELECT city, state FROM locations ORDER BY city;")

print("Analytics — avg price by city:")
run_sql('''
SELECT l.city, ROUND(AVG(s.sale_price), 2) AS avg_price
FROM sales s
JOIN homes h ON s.home_id=h.home_id
JOIN locations l ON h.location_id=l.location_id
GROUP BY l.city
ORDER BY avg_price DESC;
''')

print("Analytics — top agents by total sales:")
run_sql('''
SELECT a.name, ROUND(SUM(s.sale_price), 2) AS total_sales
FROM sales s
JOIN agents a ON s.agent_id=a.agent_id
GROUP BY a.name
ORDER BY total_sales DESC;
''')

## Wrap-Up & Next Steps

**Where to go from here**
- Try migrating the schema to PostgreSQL and compare SQL dialect differences.
- Add indexes and measure query performance.
- Create views/materialized views for common analytics.
- Explore Python ETL with `pandas` + `to_sql` pipelines into SQLite/Postgres.